In [24]:
import pandas as pd
from scipy.interpolate import interp1d
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import price_data as price
import mplfinance as mpf
import matplotlib.dates as mpl_dates
import chart

matic=price.get_price_data('1d', symbol='MATIC/USD')

# Creating Heikin Ashi candles
First we must download the candlestick data

In [25]:
matic["Date"] = pd.to_datetime(matic["unix"], unit="ms")
weekly_candles_original=price.get_price_data('1w',data=matic)
weekly_candles = weekly_candles_original.copy()
weekly_candles["Date"] = pd.to_datetime(weekly_candles["unix"], unit="ms")
weekly_candles.columns = ['Unix', 'Close', 'High', 'Low', 'Open', 'Date']
weekly_candles.set_index('Date', inplace=True)

In [26]:
matic

,unix,close,high,low,open,Date
0,1604880000000,0.015648,0.016787,0.015345,0.015759,2020-11-09
1,1604966400000,0.017288,0.017511,0.015444,0.015648,2020-11-10
2,1605052800000,0.016634,0.017796,0.016541,0.017288,2020-11-11
3,1605139200000,0.015934,0.017307,0.015752,0.016634,2020-11-12
4,1605225600000,0.016888,0.017152,0.015829,0.015934,2020-11-13
...,...,...,...,...,...,...
630,1659312000000,0.891789,0.946574,0.878667,0.928427,2022-08-01
631,1659398400000,0.876707,0.906333,0.849075,0.891789,2022-08-02
632,1659484800000,0.888095,0.926032,0.856589,0.876707,2022-08-03
633,1659571200000,0.891468,0.912140,0.875723,0.888095,2022-08-04


In [27]:
weekly_candles

,Unix,Close,High,Low,Open
Date,,,,,
2020-11-09,1604880000000,0.016412,0.017796,0.015345,0.015759
2020-11-16,1605484800000,0.019148,0.020132,0.016028,0.016412
2020-11-23,1606089600000,0.018812,0.023811,0.015837,0.019148
2020-11-30,1606694400000,0.019565,0.021083,0.017866,0.018812
2020-12-07,1607299200000,0.018132,0.019715,0.016508,0.019565
...,...,...,...,...,...
2022-07-04,1656892800000,0.571178,0.619405,0.450788,0.461359
2022-07-11,1657497600000,0.752759,0.789547,0.520142,0.571178
2022-07-18,1658102400000,0.879748,0.988154,0.745050,0.752759


Then we define a function to convert each candle stick into Heikin Ashi Candlestick

In [28]:
def heikin_ashi(previous_open, previous_close, candle):
    # print(candle)
    price_values = candle[["open","high","low","close"]]
    close = np.mean(price_values)
    open_price = 0.5*(previous_open+previous_close)
    high=max([max(price_values), open_price,close])
    low = min([min(price_values), open_price,close])
    
    return candle["unix"], open_price, high, close, low

def convert_data_to_heikin_ashi(data):
    timestamps, opens, closes, highs, lows  = [0], [0],[0],[0],[0]
    #initialise values
    for i in range(1,len(data)):
        timestamp, open_price, high, close, low = heikin_ashi(opens[-1], closes[-1],data.iloc[i])
        timestamps.append(timestamp)
        opens.append(open_price)
        highs.append(high)
        lows.append(low)
        closes.append(close)

    return pd.DataFrame({'unix':timestamps,'open':opens,'high':highs,'low':lows,'close':closes}).sort_values(by=['unix'], ignore_index=True)
    


Then we convert weekly candles and observe

In [29]:
weekly_candles
# convert_data_to_heikin_ashi(weekly_candles)

,Unix,Close,High,Low,Open
Date,,,,,
2020-11-09,1604880000000,0.016412,0.017796,0.015345,0.015759
2020-11-16,1605484800000,0.019148,0.020132,0.016028,0.016412
2020-11-23,1606089600000,0.018812,0.023811,0.015837,0.019148
2020-11-30,1606694400000,0.019565,0.021083,0.017866,0.018812
2020-12-07,1607299200000,0.018132,0.019715,0.016508,0.019565
...,...,...,...,...,...
2022-07-04,1656892800000,0.571178,0.619405,0.450788,0.461359
2022-07-11,1657497600000,0.752759,0.789547,0.520142,0.571178
2022-07-18,1658102400000,0.879748,0.988154,0.745050,0.752759


Shift timestamps up one, delete the top and bottom row (top row is null, bottom row is incomplete). The candle at each timestamp represents the previous week, any data we observe at time=t is from the past.

In [30]:
def convert_data_to_heikin_ashi_processed(data):
    timestamps, opens, closes, highs, lows  = [0], [0],[0],[0],[0]
    #initialise values
    for i in range(1,len(data)):
        timestamp, open_price, high, close, low = heikin_ashi(opens[-1], closes[-1],data.iloc[i])
        timestamps.append(timestamp)
        opens.append(open_price)
        highs.append(high)
        lows.append(low)
        closes.append(close)

    candles = pd.DataFrame({'unix':timestamps,'ha_Open':opens,'ha_High':highs,'ha_Low':lows,'ha_Close':closes}).sort_values(by=['unix'], ignore_index=True)
    timestamp_df=candles['unix'].shift(periods=-1)
    candles["unix"] = timestamp_df
    candles["Date"] = pd.to_datetime(candles["unix"], unit="ms")
    candles["Green"] = candles["ha_Close"]>candles["ha_Open"]
    candles.drop([0,1,len(candles)-1], inplace=True)
    candles.set_index("Date", inplace=True)
    candles.drop("unix", axis=1, inplace=True)
    return candles

heikin_ashi_candles = convert_data_to_heikin_ashi_processed(weekly_candles_original)
# weekly_candles.to_csv('weekly.csv')
# heikin_ashi_candles.to_csv('weekly_ha.csv')
heikin_ashi_candles


,ha_Open,ha_High,ha_Low,ha_Close,Green
Date,,,,,
2020-11-30,0.008965,0.023811,0.008965,0.019402,True
2020-12-07,0.014183,0.021083,0.014183,0.019332,True
2020-12-14,0.016758,0.019715,0.016508,0.018480,True
2020-12-21,0.017619,0.020938,0.017619,0.019060,True
2020-12-28,0.018340,0.019952,0.014667,0.018026,False
...,...,...,...,...,...
2022-07-04,0.501975,0.596350,0.416250,0.508085,True
2022-07-11,0.505030,0.619405,0.450788,0.525683,True
2022-07-18,0.515356,0.789547,0.515356,0.658406,True


Now we plot the heikin ashi candles using the mpfinance module.

In [31]:
# weekly_candles.drop(labels=["unix"], inplace=True)
# heikin_ashi_candles.drop(labels=["unix"], inplace=True)
# heikin_ashi_candles=heikin_ashi_candles.drop(["unix"], axis=1)
trimmed_weekly=weekly_candles[3:]
combnined_df=pd.concat([trimmed_weekly, heikin_ashi_candles], axis=1)
testing_df_weekly = combnined_df.loc[:,["Open", "Green"]]
testing_df_weekly

,Open,Green
Date,,
2020-11-30,0.018812,True
2020-12-07,0.019565,True
2020-12-14,0.018132,True
2020-12-21,0.019195,True
2020-12-28,0.018289,False
...,...,...
2022-07-04,0.461359,True
2022-07-11,0.571178,True
2022-07-18,0.752759,True


Note: We will assess action on the close of the heikin ashi candle, say timestamp t, at which the current price will be the open of the normal candle at timestamp t.

In [32]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(x=heikin_ashi_candles.index,
                open=heikin_ashi_candles['ha_Open'],
                high=heikin_ashi_candles['ha_High'],
                low=heikin_ashi_candles['ha_Low'],
                close=heikin_ashi_candles['ha_Close'])])

fig2 = go.Figure(data=[go.Candlestick(x=trimmed_weekly.index,
                open=trimmed_weekly['Open'],
                high=trimmed_weekly['High'],
                low=trimmed_weekly['Low'],
                close=trimmed_weekly['Close'])])

# fig.update_yaxes(type="log")
# fig2.update_yaxes(type="log")
fig.show()
fig2.show()


For each candle calculate the following metrics:
-  Close/low
-  close/high
-  close/open

In [33]:
# closes = np.array(heikin_ashi_candles["Close"])
# open_prices = np.array(heikin_ashi_candles["Open"])
# highs = np.array(heikin_ashi_candles["High"])
# lows = np.array(heikin_ashi_candles["Low"])

# c_l = np.divide(closes,lows)
# c_h = np.divide(closes, highs)
# c_o = np.divide(closes, open_prices)

In [34]:
daily = convert_data_to_heikin_ashi_processed(matic)
matic["Date"] = pd.to_datetime(matic["unix"], unit="ms")
matic.set_index('Date', inplace=True)
combined_data=pd.concat([matic, daily], axis=1).dropna()
combined_data

,unix,close,high,low,open,ha_Open,ha_High,ha_Low,ha_Close,Green
Date,,,,,,,,,,
2020-11-12,1605139200000,0.015934,0.017307,0.015752,0.016634,0.008236,0.017796,0.008236,0.017065,True
2020-11-13,1605225600000,0.016888,0.017152,0.015829,0.015934,0.012651,0.017307,0.012651,0.016407,True
2020-11-14,1605312000000,0.016685,0.016893,0.015978,0.016888,0.014529,0.017152,0.014529,0.016451,True
2020-11-15,1605398400000,0.016412,0.017086,0.015869,0.016685,0.015490,0.016893,0.015490,0.016611,True
2020-11-16,1605484800000,0.016640,0.016935,0.016028,0.016412,0.016050,0.017086,0.015869,0.016513,True
...,...,...,...,...,...,...,...,...,...,...
2022-08-01,1659312000000,0.891789,0.946574,0.878667,0.928427,0.923956,1.016469,0.911341,0.946822,True
2022-08-02,1659398400000,0.876707,0.906333,0.849075,0.891789,0.935389,0.946574,0.878667,0.911364,False
2022-08-03,1659484800000,0.888095,0.926032,0.856589,0.876707,0.923377,0.923377,0.849075,0.880976,False


In [35]:
fig = go.Figure(data=[go.Candlestick(x=daily.index,
                open=daily['ha_Open'],
                high=daily['ha_High'],
                low=daily['ha_Low'],
                close=daily['ha_Close'])])

fig2 = go.Figure(data=[go.Candlestick(x=matic.index,
                open=matic['open'],
                high=matic['high'],
                low=matic['low'],
                close=matic['close'])])

fig.show()
fig2.show()

### Backtesting
Now that we have matched the heikin ashi candles to the normal candles, we can backtest a simple trading strategy that sells when the HA candle is red and buys when it's green.

In [36]:
def single_test(raw_data: pd.DataFrame, sample_size=None):
    normal_candles = raw_data.__copy__()
    ha = convert_data_to_heikin_ashi_processed(raw_data)
    normal_candles["Date"] = pd.to_datetime(normal_candles["unix"], unit="ms")
    normal_candles.columns = ['Unix', 'Close', 'High', 'Low', 'Open', 'Date']
    normal_candles.set_index('Date', inplace=True)
    combined_data=pd.concat([normal_candles, ha], axis=1).dropna()
    test_df = combined_data.loc[:,["Open", "Green"]]
    if sample_size !=None:
        test_df = test_df.iloc[len(test_df)-sample_size-1:]
    

    longs, shorts, outcome, profit, final_equity, equity_curve=backtest_heikin_ashi(test_df)
    fig = go.Figure(data=[go.Candlestick(x=combined_data.index,
                    open=combined_data['Open'],
                    high=combined_data['High'],
                    low=combined_data['Low'],
                    close=combined_data['Close'])])
    
    fig2 = go.Figure(data=[go.Candlestick(x=combined_data.index,
                    open=combined_data['ha_Open'],
                    high=combined_data['ha_High'],
                    low=combined_data['ha_Low'],
                    close=combined_data['ha_Close'])])
    # plt.vlines(longs, ymin=raw_data['low'].min(), ymax=raw_data['high'].max(), colors='green')
    # plt.vlines(shorts, ymin=raw_data['low'].min(), ymax=raw_data['high'].max(), colors='red')
    print(np.sum(outcome)/len(outcome))
    print(np.mean(profit))
    print(final_equity)
    return longs, shorts, outcome, profit, final_equity, equity_curve, fig, fig2

def single_test_lower_timeframe(raw_data: pd.DataFrame, no_consecutive, sample_size=None):
    normal_candles = raw_data.__copy__()
    ha = convert_data_to_heikin_ashi_processed(raw_data)
    normal_candles["Date"] = pd.to_datetime(normal_candles["unix"], unit="ms")
    normal_candles.columns = ['Unix', 'Close', 'High', 'Low', 'Open', 'Date']
    normal_candles.set_index('Date', inplace=True)
    combined_data=pd.concat([normal_candles, ha], axis=1).dropna()
    test_df = combined_data.loc[:,["Open", "Green"]]
    if sample_size !=None:
        test_df = test_df.iloc[len(test_df)-sample_size-1:]
    
    longs, shorts, outcome, profit, final_equity, equity_curve=backtest_heikin_ashi_lower_timeframe(test_df, no_consecutive)
    fig = go.Figure(data=[go.Candlestick(x=combined_data.index,
                    open=combined_data['Open'],
                    high=combined_data['High'],
                    low=combined_data['Low'],
                    close=combined_data['Close'])])
    
    fig2 = go.Figure(data=[go.Candlestick(x=combined_data.index,
                    open=combined_data['ha_Open'],
                    high=combined_data['ha_High'],
                    low=combined_data['ha_Low'],
                    close=combined_data['ha_Close'])])
    # plt.vlines(longs, ymin=raw_data['low'].min(), ymax=raw_data['high'].max(), colors='green')
    # plt.vlines(shorts, ymin=raw_data['low'].min(), ymax=raw_data['high'].max(), colors='red')
    print(np.sum(outcome)/len(outcome))
    print(np.mean(profit))
    print(final_equity)
    return longs, shorts, outcome, profit, final_equity, equity_curve, fig, fig2

    

def backtest_heikin_ashi(test_df): # raw data contains normal OHCL candles
    equity=1
    fee_rate=0.0006
    state='neutral'
    timestamps = test_df.index.date
    equity_record=[]
    longs=[]
    shorts=[]
    outcome=[]
    profit=[]
    sl=None
    entry=None
    equity_curve=[]
    print('Datapoints: ',len(test_df))
    for i in range(1,len(test_df)):

        time=timestamps[i]
        
        current=test_df.iloc[i]['Open'].item() # uses opens now as opens are confirmed
        green= test_df.iloc[i]["Green"]
        
        if green and state != 'long':
            #remember to calculate profit if flipping from short
            if state=='short':
                outcome.append(current<entry)
                profit.append(1-(current/entry))
                equity=equity*(1+(1-(current/entry))-fee_rate)
                # equity=(entry*trade_amount-current*trade_amount)+equity
            entry=current
            equity_curve.append([time,equity])
            state='long'
            longs.append(time)
        elif not(green) and state != 'short':
            if state=='long':
                outcome.append(current>entry)
                profit.append(current/entry -1)
                equity=equity*(1+(current/entry -1)-fee_rate)
            entry=current
            equity_curve.append([time,equity])    
            # trade_amount=equity*entry
            state='short'
            shorts.append(time)
            #print(state+' from $'+str(current))
        if equity < 0.05:
            equity=0
            print('went broke')
            break

        if i==len(test_df)-1:
            if state=='long':
                equity=equity*(current/entry)
            elif state=='short':
                equity=equity*(2-current/entry)
            equity_curve.append([time,equity])

    
    return longs,shorts,np.array(outcome), np.array(profit), equity, np.array(equity_curve)

def backtest_heikin_ashi_lower_timeframe(test_df, no_consecutive): # raw data contains normal OHCL candles
    equity=1
    fee_rate=0.0006
    state='neutral'
    timestamps = test_df.index.date
    equity_record=[]
    longs=[]
    shorts=[]
    outcome=[]
    profit=[]
    sl=None
    entry=None
    equity_curve=[]
    print('Datapoints: ',len(test_df))
    for i in range(1,len(test_df)):

        time=timestamps[i]
        
        current=test_df.iloc[i]['Open'].item() # uses opens now as opens are confirmed
        green= test_df.iloc[i-no_consecutive:i+1]["Green"].all()
        
        if green and state != 'long':
            #remember to calculate profit if flipping from short
            if state=='short':
                outcome.append(current<entry)
                profit.append(1-(current/entry))
                equity=equity*(1+(1-(current/entry))-fee_rate)
                # equity=(entry*trade_amount-current*trade_amount)+equity
            entry=current
            equity_curve.append([time,equity])
            state='long'
            longs.append(time)
        elif not(green) and state != 'short':
            if state=='long':
                outcome.append(current>entry)
                profit.append(current/entry -1)
                equity=equity*(1+(current/entry -1)-fee_rate)
            entry=current
            equity_curve.append([time,equity])    
            # trade_amount=equity*entry
            state='short'
            shorts.append(time)
            #print(state+' from $'+str(current))
        if equity < 0.05:
            equity=0
            print('went broke')
            break

        if i==len(test_df)-1:
            if state=='long':
                equity=equity*(current/entry)
            elif state=='short':
                equity=equity*(2-current/entry)
            equity_curve.append([time,equity])

    
    return longs,shorts,np.array(outcome), np.array(profit), equity, np.array(equity_curve)

longs, shorts, outcome, profit, final_equity, equity_curve, fig, fig2 = single_test(weekly_candles_original)
# fig.add_vline(x=longs[0], line_width=3, line_dash="dash", line_color="green")
fig.show()
fig2.show()
print(equity_curve)

In [38]:
longs2, shorts2, outcome2, profit2, final_equity2, equity_curve2, fig, fig2 = single_test_lower_timeframe(matic, 1)

Datapoints:  632
0.432
0.024802425880018207
2.4519663336204083


In [ ]:
hourly = price.get_price_data('1h',symbol='ETH/USD' )
hourly
# longs, shorts, outcome, profit, final_equity, equity_curve, fig = single_test(hourly)

,unix,close,high,low,open
0,1640552400000,4085.1,4107.3,4044.2,4074.0
1,1640556000000,4084.6,4095.7,4079.5,4085.1
2,1640559600000,4066.2,4093.7,4061.2,4084.6
3,1640563200000,4063.3,4081.8,4059.8,4066.2
4,1640566800000,4077.0,4080.8,4049.5,4063.3
...,...,...,...,...,...
1496,1645938000000,2709.8,2724.5,2698.3,2706.9
1497,1645941600000,2733.5,2739.0,2709.3,2709.8
1498,1645945200000,2737.4,2745.4,2733.5,2733.5
1499,1645948800000,2743.4,2748.3,2723.5,2737.4
